In [ ]:
import os
import numpy as np
from PIL import Image, ImageChops
from scipy import ndimage
import matplotlib.pyplot as plt

# === Step 1: Load the image safely ===
img_path = 'C:/Users/hp/Downloads/IMD/difference53.jpg'  # ✅ FIXED path

if not os.path.isfile(img_path):
    raise FileNotFoundError(f"File not found at: {img_path}")

# Convert to RGB
img = Image.open(img_path).convert('RGB')

# === Step 2: Convert to grayscale and then to black & white using threshold ===
gray_img = img.convert('L')  # convert to grayscale

threshold = 50
bw_img = gray_img.point(lambda x: 255 if x > threshold else 0, mode='1')

# Save black & white version
bw_path = 'C:/Users/hp/Downloads/IMD/image_bw_01.jpg'
bw_img.save(bw_path)

# === Step 3: Convert to binary numpy array ===
mask = np.array(bw_img, dtype=np.uint8)

# === Step 4: Label connected components ===
labels, n = ndimage.label(mask, structure=[[0, 1, 0],
                                           [1, 1, 1],
                                           [0, 1, 0]])

if n == 0:
    raise ValueError("No white pixels found at all.")

# === Step 5: Find the component with largest width & line-like height ===
best_label = None
best_width = -1
slices = ndimage.find_objects(labels)

for i, sl in enumerate(slices, start=1):
    if sl is None:
        continue
    y0, y1 = sl[0].start, sl[0].stop
    x0, x1 = sl[1].start, sl[1].stop
    width = x1 - x0
    height = y1 - y0
    if width > best_width and height <= 10:  # tweak if needed
        best_label = i
        best_width = width
        crop_box = (x0, y0, x1, y1)

if best_label is None:
    raise ValueError("Couldn’t find a continuous line – try relaxing height check.")

# === Step 6: Create mask for best component ===
line_mask = (labels == best_label).astype(np.uint8) * 255
line_mask_img = Image.fromarray(line_mask, mode='L')
line_mask_path = 'C:/Users/hp/Downloads/IMD/white_line_mask_01.jpg'
line_mask_img.save(line_mask_path)

# === Step 7: Crop just the white line from original B/W ===
cropped_line = bw_img.crop(crop_box)
cropped_line_path = 'C:/Users/hp/Downloads/IMD/white_line_only_01.jpg'
cropped_line.save(cropped_line_path)

# === Step 8: Display Results ===
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(bw_img, cmap='gray')
plt.title("1. Original B/W Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(line_mask_img, cmap='gray')
plt.title("2. Detected White Line Mask")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cropped_line, cmap='gray')
plt.title("3. Cropped White Line Only")
plt.axis('off')

plt.tight_layout()
plt.show()

# === Optional Output ===
print("Line mask saved at:", line_mask_path)
print(" Cropped white line saved at:", cropped_line_path)
